# Module 01b: SGD & Normal Equation

**CS229 Aligned Curriculum** | *Gold Standard Edition*

**CS229 Aligned Curriculum** | *Gold Standard Edition*

## 🎯 TARGET PEMBELAJARAN

| No | Skill | Cara Verifikasi |
|---|---|---|
| 1 | Bandingkan **Batch vs SGD** | Tahu pros/cons |
| 2 | Derive **Normal Equation** | Step-by-step |
| 3 | Paham **probabilistic interpretation** | Kenapa MSE? |
| 4 | Paham **MLE = Least Squares** | Derive koneksi |

---

## 🎭 Analogi untuk Pemula: Mencari Titik Terendah di Lembah

### 🌟 Level 1: Berjalan Menuruni Bukit

Bayangkan Anda tersesat di pegunungan pada malam hari. Tujuan Anda: **mencapai lembah terendah**.

**1. Batch Gradient Descent (Hati-hati Banget):**
Anda berhenti setiap langkah, cek seluruh peta (semua data), baru tentukan arah langkah berikutnya.
- **Pro:** Akurat, pasti sampai.
- **Con:** Lambat banget! Capek cek peta terus.

**2. Stochastic GD (Nekat):**
Anda langsung jalan berdasarkan satu jejak kaki di depan Anda (satu data point).
- **Pro:** Cepat banget!
- **Con:** Jalannya zig-zag, kadang malah naik sebentar.

**3. Mini-Batch GD (Kompromi):**
Anda cek beberapa jejak kaki terdekat (batch kecil), baru tentukan arah.
- **Pro:** Cukup cepat, cukup akurat.
- **Con:** Harus pilih ukuran batch yang pas.

**4. Momentum (Pakai Skateboard):**
Anda pakai skateboard, jadi kecepatan sebelumnya membawa Anda terus. Tidak berhenti tiba-tiba.
- **Pro:** Lebih cepat melewati dataran datar, tidak terjebak lubang kecil.
- **Con:** Bisa kebablasan kalau turunan curam.

---

### 📖 Istilah Kunci untuk Pemula

| Istilah | Arti Sederhana |
| :--- | :--- |
| **Learning Rate (α)** | Seberapa besar langkah kaki Anda. Terlalu besar = melompat-lompat. Terlalu kecil = lama sampai. |
| **Epoch** | Satu kali jalan melewati SELURUH data training. |
| **Batch Size** | Berapa data yang Anda lihat sebelum mengambil satu langkah. |
| **Convergence** | Sudah sampai di titik terendah (atau cukup dekat). |
| **Cost/Loss** | Seberapa jauh Anda dari tujuan. Makin kecil, makin bagus. |


## 🚀 PART 1: STOCHASTIC GRADIENT DESCENT

### 1.1 Update Rule

$$\theta_j := \theta_j - \alpha (h_\theta(x^{(i)}) - y^{(i)}) x_j^{(i)}$$

#### 🗣️ CARA BACA:

> "Theta j di-update dengan mengurangi alpha kali error untuk data i kali xj i"

### 1.2 Perbandingan

| Aspek | Batch GD | SGD |
|---|---|---|
| Data per update | Semua m | 1 |
| Speed per update | LAMBAT | CEPAT |
| Convergence | Smooth | Noisy (zigzag) |
| Memori | O(m) | O(1) |

### 1.3 Visualisasi Convergence

```
BATCH GD:                    SGD:

J(θ)                         J(θ)
 │\                           │
 │ \                          │ \    /\
 │  \                         │  \  /  \/\
 │   \                        │   \/      \/\
 │    \___                    │           \___
 │        \___                │               
 └─────────────→ iter         └─────────────→ iter

Smooth descent                Noisy tapi lebih cepat 
                              sampai ke area minimum
```

### 1.4 Mini-Batch: Kompromi

```
for each mini-batch of size B:
    gradient = (1/B) Σ_batch (error × x)
    θ = θ - α × gradient
```

Biasanya B = 32, 64, 128, 256

---

## 🧮 PART 2: NORMAL EQUATION (Closed-Form Solution)

### 2.1 Ide

Daripada iterate dengan gradient descent, langsung SOLVE!

Set gradient = 0, solve for θ.

### 2.2 Matrix Notation

Design matrix X (m × n+1):
$$X = \begin{bmatrix} — (x^{(1)})^T — \\ — (x^{(2)})^T — \\ \vdots \\ — (x^{(m)})^T — \end{bmatrix}$$

Target vector y (m × 1):
$$y = \begin{bmatrix} y^{(1)} \\ y^{(2)} \\ \vdots \\ y^{(m)} \end{bmatrix}$$

### 2.3 Cost Function dalam Matrix Form

$$J(\theta) = \frac{1}{2m}(X\theta - y)^T(X\theta - y)$$

### 2.4 Derivasi Normal Equation (Step-by-Step)

**STEP 1:** Tulis J tanpa 1/2m (tidak mempengaruhi optimal θ)

$$J = \frac{1}{2}(X\theta - y)^T(X\theta - y)$$

**STEP 2:** Expand

$$J = \frac{1}{2}(\theta^T X^T X \theta - 2\theta^T X^T y + y^T y)$$

**STEP 3:** Gradient w.r.t. θ

Gunakan aturan:
- $\nabla_\theta (\theta^T A \theta) = 2A\theta$ (jika A simetrik)
- $\nabla_\theta (b^T \theta) = b$

$$\nabla_\theta J = \frac{1}{2}(2X^T X \theta - 2X^T y) = X^T X \theta - X^T y$$

**STEP 4:** Set = 0 dan solve

$$X^T X \theta = X^T y$$

$$\theta = (X^T X)^{-1} X^T y$$

### 2.5 Normal Equation

$$\boxed{\theta = (X^T X)^{-1} X^T y}$$

#### 🗣️ CARA BACA:

> "Theta sama dengan X transpose X inverse dikali X transpose y"

### 2.6 Kapan Pakai Normal Eq vs GD?

| Normal Equation | Gradient Descent |
|---|---|
| n kecil (< 10,000) | n besar |
| No need to choose α | Perlu tune α |
| O(n³) untuk inverse | O(kn²) per iteration |
| XᵀX harus invertible | Selalu work |

---

## 📊 PART 3: PROBABILISTIC INTERPRETATION

### 3.1 Pertanyaan

Kenapa kita pakai **squared error**? Kenapa bukan |error| atau error⁴?

**Jawaban:** Dari perspektif probabilistik!

### 3.2 Assumption

Asumsikan:
$$y^{(i)} = \theta^T x^{(i)} + \epsilon^{(i)}$$

Dimana $\epsilon^{(i)} \sim \mathcal{N}(0, \sigma^2)$ (Gaussian noise)

#### 🗣️ CARA BACA:

> "y sama dengan theta transpose x plus noise epsilon, dimana epsilon adalah Gaussian dengan mean 0"

### 3.3 Likelihood

$$p(y^{(i)} | x^{(i)}; \theta) = \frac{1}{\sqrt{2\pi}\sigma} \exp\left(-\frac{(y^{(i)} - \theta^T x^{(i)})^2}{2\sigma^2}\right)$$

### 3.4 Log-Likelihood

$$\ell(\theta) = \sum_{i=1}^{m} \log p(y^{(i)} | x^{(i)}; \theta)$$

$$= \sum_{i=1}^{m} \left[ -\log(\sqrt{2\pi}\sigma) - \frac{(y^{(i)} - \theta^T x^{(i)})^2}{2\sigma^2} \right]$$

$$= m \cdot C - \frac{1}{2\sigma^2} \sum_{i=1}^{m} (y^{(i)} - \theta^T x^{(i)})^2$$

### 3.5 Maximize Log-Likelihood = Minimize MSE!

$$\max_\theta \ell(\theta) \Leftrightarrow \min_\theta \sum_{i=1}^{m} (h_\theta(x^{(i)}) - y^{(i)})^2$$

> 💡 **MLE dengan Gaussian noise = Least Squares!**

Ini justifikasi probabilistik kenapa pakai squared error!

## 🛠️ PART 4: IMPLEMENTASI ROBUST (SGD, Mini-Batch, Momentum)

Kita tidak hanya akan membuat loop sederhana. Kita akan implementasi Class yang siap produksi dengan:
1.  **Feature Scaling (StandardScaler)**: Wajib untuk SGD agar konvergensi cepat.
2.  **Mini-Batch Support**: Kompromi terbaik antara kecepatan dan stabilitas.
3.  **Momentum**: Teknik mempercepat SGD menembus "lembah" landai.
4.  **Learning Rate Decay**: Mengecilkan langkah saat mendekati optimum.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Generate Synthetic Data (Regression)
np.random.seed(42)
m = 100
X = 2 * np.random.rand(m, 1)
y = 4 + 3 * X + np.random.randn(m, 1) # y = 4 + 3x + noise

# Outlier Extreme (untuk demo robustness)
X_outlier = np.append(X, [[0.5], [1.5]]).reshape(-1, 1)
y_outlier = np.append(y, [[20], [20]]).reshape(-1, 1) # Outliers

plt.scatter(X, y, alpha=0.6)
plt.xlabel("Input Feature (x)")
plt.ylabel("Target (y)")
plt.title("Generated Dataset")
plt.show()


### 4.1 The `AdvancedSGD` Class
Class ini membungkus semua logika SGD modern.


In [ ]:
class AdvancedSGD:
    def __init__(self, learning_rate=0.01, n_epochs=50, batch_size=None, momentum=0.0, decay_rate=0.0):
        self.lr = learning_rate
        self.epochs = n_epochs
        self.batch_size = batch_size
        self.momentum = momentum
        self.decay = decay_rate
        self.theta = None
        self.history = [] # To store cost per epoch
        
    def _compute_cost(self, X, y, theta):
        m = len(y)
        predictions = X.dot(theta)
        cost = (1/(2*m)) * np.sum(np.square(predictions - y))
        return cost

    def fit(self, X, y):
        m, n = X.shape
        X_b = np.c_[np.ones((m, 1)), X] # Add bias term
        self.theta = np.random.randn(n + 1, 1) # Random initialization
        velocity = np.zeros_like(self.theta) # For momentum
        
        initial_lr = self.lr
        
        self.theta_path = [] # Store path for visualization
        self.theta_path.append(self.theta.copy())

        for epoch in range(self.epochs):
            # Shuffle data
            shuffled_indices = np.random.permutation(m)
            X_b_shuffled = X_b[shuffled_indices]
            y_shuffled = y[shuffled_indices]
            
            # Determine Batch Size
            if self.batch_size is None:
                bs = m # Batch GD (Full dataset)
            else:
                bs = self.batch_size # Stochastic/Mini-batch
            
            for i in range(0, m, bs):
                xi = X_b_shuffled[i:i+bs]
                yi = y_shuffled[i:i+bs]
                
                gradients = 2/bs * xi.T.dot(xi.dot(self.theta) - yi)
                
                # Momentum Logic
                velocity = self.momentum * velocity + self.lr * gradients
                self.theta = self.theta - velocity
                
                self.theta_path.append(self.theta.copy())
            
            # Learning Rate Decay
            self.lr = initial_lr / (1 + self.decay * epoch)
            
            cost = self._compute_cost(X_b, y, self.theta)
            self.history.append(cost)
            
        return self

    def predict(self, X):
        X_b = np.c_[np.ones((len(X), 1)), X]
        return X_b.dot(self.theta)



### 4.2 Visualisasi: Battle of Optimizers
Kita bandingkan jalur pergerakan **Batch GD**, **SGD**, dan **Mini-Batch** menuju Minimum.


In [ ]:
# Helper untuk Contour Plot
def plot_optimization_path(models, names, X, y):
    # Setup Grid untuk Theta0 dan Theta1
    t0 = np.linspace(3, 5, 100) # Sekitar true bias 4
    t1 = np.linspace(2, 4, 100) # Sekitar true weight 3
    T0, T1 = np.meshgrid(t0, t1)
    J_grid = np.zeros(T0.shape)
    
    # Hitung cost untuk setiap titik grid
    X_b = np.c_[np.ones((len(X), 1)), X]
    for i in range(T0.shape[0]):
        for j in range(T0.shape[1]):
            theta_temp = np.array([[T0[i,j]], [T1[i,j]]])
            J_grid[i,j] = (1/(2*len(y))) * np.sum((X_b.dot(theta_temp) - y)**2)

    plt.figure(figsize=(10, 6))
    plt.contourf(T0, T1, J_grid, levels=20, cmap='viridis', alpha=0.6)
    plt.colorbar(label='Cost Function J')
    
    colors = ['r', 'g', 'b', 'orange']
    markers = ['o', '+', 'x', 's']
    
    for idx, model in enumerate(models):
        path = np.array(model.theta_path)
        # Plot path
        plt.plot(path[:, 0], path[:, 1], '-', color=colors[idx], linewidth=1, label=names[idx])
        plt.plot(path[:, 0], path[:, 1], markers[idx], color=colors[idx], markersize=3)
        # Plot start and end
        plt.plot(path[-1, 0], path[-1, 1], 'k*', markersize=10) # End point

    plt.xlabel(r'$	heta_0$ (Bias)')
    plt.ylabel(r'$	heta_1$ (Weight)')
    plt.title('Path of Gradient Descent Variants (Contour Plot)')
    plt.legend()
    plt.show()

# Run Models to Compare
# 1. Batch GD (Batch size = m)
bgd = AdvancedSGD(learning_rate=0.1, n_epochs=50, batch_size=m)
bgd.fit(X, y)

# 2. Stochastic GD (Batch size = 1)
sgd = AdvancedSGD(learning_rate=0.1, n_epochs=50, batch_size=1) # Noisy!
sgd.fit(X, y)

# 3. Mini-Batch GD (Batch size = 20)
mbgd = AdvancedSGD(learning_rate=0.1, n_epochs=50, batch_size=20)
mbgd.fit(X, y)

# 4. Momentum GD (Batch size = 20 + Momentum 0.9)
mom = AdvancedSGD(learning_rate=0.1, n_epochs=50, batch_size=20, momentum=0.9)
mom.fit(X, y)

plot_optimization_path([bgd, sgd, mbgd, mom], 
                      ['Batch GD', 'SGD', 'Mini-Batch', 'Momentum'], 
                      X, y)



### 🔄 Interpretasi Grafik
1.  **Batch GD (Merah):** Jalan lurus, smooth, tapi lambat per epoch karena hitung semua data.
2.  **SGD (Hijau):** Sangat "noisy" (zig-zag). Karena update tiap 1 data, dia 'mabuk' tapi rata-rata menuju arah benar.
3.  **Mini-Batch (Biru):** Trade-off terbaik. Agak noisy tapi lebih stabil dari SGD, dan lebih efisien komputasi.
4.  **Momentum (Orange):** Meluncur cepat! Momentum membantu menembus area landai dan meredam osilasi.



In [ ]:
# ═══════════════════════════════════════════════════════════════
# 📊 BENCHMARK: PERFORMANCE COMPARISON TABLE
# ═══════════════════════════════════════════════════════════════
import time

# Define X_b for this cell
X_b = np.c_[np.ones((len(X), 1)), X]

# Re-train with timing
results = []
configs = [
    ('Batch GD', {'batch_size': m, 'momentum': 0.0}),
    ('Stochastic GD', {'batch_size': 1, 'momentum': 0.0}),
    ('Mini-Batch GD', {'batch_size': 16, 'momentum': 0.0}),
    ('Momentum SGD', {'batch_size': 16, 'momentum': 0.9}),
]

for name, cfg in configs:
    model = AdvancedSGD(learning_rate=0.01, n_epochs=100, **cfg)
    start = time.time()
    model.fit(X_b, y)
    elapsed = (time.time() - start) * 1000  # ms
    results.append({
        'Method': name,
        'Time (ms)': f'{elapsed:.2f}',
        'Final Cost': f'{model.history[-1]:.6f}',
        'θ₀': f'{model.theta[0][0]:.4f}',
        'θ₁': f'{model.theta[1][0]:.4f}'
    })

print('\n📊 BENCHMARK RESULTS')
print('='*70)
print(f'{"Method":<16} | {"Time (ms)":<10} | {"Final Cost":<12} | {"θ₀":<8} | {"θ₁":<8}')
print('-'*70)
for r in results:
    print(f'{r["Method"]:<16} | {r["Time (ms)"]:<10} | {r["Final Cost"]:<12} | {r["θ₀"]:<8} | {r["θ₁"]:<8}')
print('='*70)
print('\n💡 Insight: Mini-Batch + Momentum biasanya memberikan trade-off terbaik.')


### 🛑 Early Stopping (Bonus)

**Konsep:** Berhenti training lebih awal jika cost sudah tidak berubah signifikan.

```python
# Di training loop:
if len(self.history) > patience:
    delta = abs(self.history[-1] - self.history[-patience])
    if delta < tolerance:
        print(f'Early stopping at epoch {epoch}')
        break
```

**Kapan Pakai:**
- Dataset besar, training lama.
- Ingin hindari overfitting.
- Resource (GPU/CPU) terbatas.


## 📏 PART 5: ROBUSTNESS & SCALING
### 5.1 Kenapa Feature Scaling Penting untuk SGD?
Jika fitur punya skala beda (misal: luas rumah 2000 vs kamar 3), kontur cost function akan lonjong (elips pipih).
Gradient Descent akan berosilasi lambat di lembah pipih itu.
**Solusi:** StandardScaler ($z = rac{x - \mu}{\sigma}$).



## 🧮 PART 6: NORMAL EQUATION EDGE CASES
Rumus Normal Equation: $\theta = (X^T X)^{-1} X^T y$

Bagaimana jika $X^T X$ **Singular** (Tidak punya invers)?
Ini terjadi jika:
1.  Fitur redundan (Linear dependency: $x_2 = 2x_1$).
2.  Fitur lebih banyak dari data ($n > m$).

Solusi: Gunakan **Pseudo-Inverse** (Moore-Penrose) -> `np.linalg.pinv` (bukan `inv`).



In [ ]:
# Demo Singular Matrix
# Buat fitur redundan
X_redundant = np.array([[1, 2], [2, 4], [3, 6], [4, 8]]) # x2 = 2*x1
y_redundant = np.array([3, 6, 9, 12])

# Coba Normal Equation Biasa (inv) - Akan Error atau Numerical Unstable
try:
    X_b_red = np.c_[np.ones((4, 1)), X_redundant]
    theta_bad = np.linalg.inv(X_b_red.T.dot(X_b_red)).dot(X_b_red.T).dot(y_redundant)
    print("Inversion Successful (Lucky):", theta_bad)
except np.linalg.LinAlgError:
    print("❌ Error: Matrix Singular! Tidak bisa di-inverse biasa.")

# Solusi: PINV (Pseudo Inverse)
theta_good = np.linalg.pinv(X_b_red.T.dot(X_b_red)).dot(X_b_red.T).dot(y_redundant)
print("✅ Solusi dengan Pseudo-Inverse (pinv):", theta_good)



## 🔍 BONUS 1: Gradient Check (Verifikasi Implementasi)

**Masalah:** Bagaimana kita tahu implementasi gradient kita benar?

**Solusi:** Bandingkan **analytical gradient** (rumus) dengan **numerical gradient** (finite difference).

$$\text{Numerical Gradient} = \frac{J(\theta + \epsilon) - J(\theta - \epsilon)}{2\epsilon}$$

Jika selisihnya < 1e-5, implementasi kita **valid**.


In [ ]:
# ═══════════════════════════════════════════════════════════════
# 🔍 GRADIENT CHECK: Verify our implementation is correct
# ═══════════════════════════════════════════════════════════════
def compute_cost(theta, X, y):
    m = len(y)
    predictions = X @ theta
    return (1/(2*m)) * np.sum((predictions - y)**2)

def numerical_gradient(theta, X, y, epsilon=1e-7):
    """Compute gradient using finite difference"""
    num_grad = np.zeros_like(theta)
    for i in range(len(theta)):
        theta_plus = theta.copy()
        theta_minus = theta.copy()
        theta_plus[i] += epsilon
        theta_minus[i] -= epsilon
        num_grad[i] = (compute_cost(theta_plus, X, y) - compute_cost(theta_minus, X, y)) / (2 * epsilon)
    return num_grad

def analytical_gradient(theta, X, y):
    """Compute gradient using formula"""
    m = len(y)
    return (1/m) * X.T @ (X @ theta - y)

# Define X_b for this cell
X_b = np.c_[np.ones((len(X), 1)), X]

# Test
theta_test = np.random.randn(2, 1)
num_grad = numerical_gradient(theta_test, X_b, y)
ana_grad = analytical_gradient(theta_test, X_b, y)

print('🔍 GRADIENT CHECK')
print('='*50)
print(f'Numerical Gradient:  {num_grad.flatten()}')
print(f'Analytical Gradient: {ana_grad.flatten()}')
print(f'Difference: {np.linalg.norm(num_grad - ana_grad):.2e}')
print('='*50)
if np.linalg.norm(num_grad - ana_grad) < 1e-5:
    print('✅ Gradient implementation is CORRECT!')
else:
    print('❌ WARNING: Gradient mismatch detected!')


## 🧮 BONUS 2: Hitung Manual 1 Epoch (Intuisi Dasar)

Mari hitung **satu langkah** Gradient Descent dengan tangan untuk 3 data points.

**Data:**
| i | $x_i$ | $y_i$ |
|---|-------|-------|
| 1 | 1 | 2 |
| 2 | 2 | 4 |
| 3 | 3 | 5 |

**Initial:** $\theta_0 = 0$, $\theta_1 = 0$, $\alpha = 0.1$


In [ ]:
# ═══════════════════════════════════════════════════════════════
# 🧮 MANUAL CALCULATION: 1 Epoch Step-by-Step
# ═══════════════════════════════════════════════════════════════
print('🧮 MANUAL GRADIENT DESCENT CALCULATION')
print('='*60)

# Tiny dataset
X_tiny = np.array([[1], [2], [3]])
y_tiny = np.array([[2], [4], [5]])
X_tiny_b = np.c_[np.ones((3, 1)), X_tiny]  # Add bias

# Initial theta
theta = np.array([[0], [0]])  # [theta0, theta1]
alpha = 0.1
m = 3

print(f'Data: X = {X_tiny.flatten()}, y = {y_tiny.flatten()}')
print(f'Initial theta: {theta.flatten()}')
print(f'Learning rate: {alpha}')
print()

# Step 1: Predictions
h = X_tiny_b @ theta
print('STEP 1: Compute predictions h = X @ theta')
print(f'  h = {h.flatten()}')
print()

# Step 2: Errors
errors = h - y_tiny
print('STEP 2: Compute errors (h - y)')
print(f'  errors = {errors.flatten()}')
print()

# Step 3: Gradient
gradient = (1/m) * X_tiny_b.T @ errors
print('STEP 3: Compute gradient = (1/m) * X.T @ errors')
print(f'  gradient = {gradient.flatten()}')
print()

# Step 4: Update
theta_new = theta - alpha * gradient
print('STEP 4: Update theta = theta - alpha * gradient')
print(f'  theta_new = {theta_new.flatten()}')
print()

print('='*60)
print('💡 Setelah 1 epoch, theta berubah dari [0, 0] ke', theta_new.flatten())
print('   Proses ini diulang ratusan/ribuan kali sampai konvergen!')


## ⚙️ BONUS 3: Feature Scaling Implementation

**Problem:** Jika fitur punya skala berbeda (misal: usia 0-100, gaji 1M-100M), GD akan lambat.

**Solution:** Standardize semua fitur ke mean=0, std=1.

$$x_{scaled} = \frac{x - \mu}{\sigma}$$


In [ ]:
# ═══════════════════════════════════════════════════════════════
# ⚙️ FEATURE SCALING: StandardScaler from scratch
# ═══════════════════════════════════════════════════════════════
class StandardScaler:
    def __init__(self):
        self.mean_ = None
        self.std_ = None
    
    def fit(self, X):
        self.mean_ = np.mean(X, axis=0)
        self.std_ = np.std(X, axis=0)
        return self
    
    def transform(self, X):
        return (X - self.mean_) / self.std_
    
    def fit_transform(self, X):
        return self.fit(X).transform(X)
    
    def inverse_transform(self, X_scaled):
        return X_scaled * self.std_ + self.mean_

# Demo
print('⚙️ FEATURE SCALING DEMO')
print('='*50)

# Create data with different scales
X_unscaled = np.array([[25, 50000], [30, 60000], [35, 80000], [40, 100000]])
print('Original data (Age, Salary):')
print(X_unscaled)
print()

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_unscaled)
print('Scaled data (mean=0, std=1):')
print(np.round(X_scaled, 3))
print()
print(f'Mean after scaling: {np.round(X_scaled.mean(axis=0), 6)}')
print(f'Std after scaling: {np.round(X_scaled.std(axis=0), 6)}')


## 🎯 BONUS 4: Learning Rate Finder

**Ide:** Naikkan learning rate secara eksponensial, plot loss-nya.

**Interpretasi:**
- LR terlalu kecil → loss turun lambat
- LR optimal → loss turun cepat
- LR terlalu besar → loss **naik** (exploding!)


In [ ]:
# ═══════════════════════════════════════════════════════════════
# 🎯 LEARNING RATE FINDER
# ═══════════════════════════════════════════════════════════════
# Define X_b and m for this cell
X_b = np.c_[np.ones((len(X), 1)), X]
m = len(X)

learning_rates = np.logspace(-5, 0, 50)  # 1e-5 to 1
losses = []

for lr in learning_rates:
    theta = np.zeros((2, 1))
    # Run 10 iterations
    for _ in range(10):
        gradient = (1/m) * X_b.T @ (X_b @ theta - y)
        theta = theta - lr * gradient
    loss = compute_cost(theta, X_b, y)
    losses.append(min(loss, 1e10))  # Cap for visualization

plt.figure(figsize=(10, 5))
plt.semilogx(learning_rates, losses, 'b-', linewidth=2)
plt.axhline(y=min(losses), color='g', linestyle='--', label=f'Best Loss: {min(losses):.4f}')
plt.xlabel('Learning Rate (log scale)')
plt.ylabel('Loss after 10 iterations')
plt.title('🎯 Learning Rate Finder')
plt.legend()
plt.grid(True, alpha=0.3)

# Find optimal LR
optimal_idx = np.argmin(losses)
optimal_lr = learning_rates[optimal_idx]
plt.axvline(x=optimal_lr, color='r', linestyle='--', label=f'Optimal LR: {optimal_lr:.4f}')
plt.legend()
plt.show()

print(f'💡 Suggested Learning Rate: {optimal_lr:.4f}')


## 📊 BONUS 5: Multi-Feature Example

Sejauh ini kita hanya pakai 1 fitur. Mari coba dengan **3 fitur**:
- Luas Tanah (m²)
- Jumlah Kamar
- Usia Bangunan (tahun)


In [ ]:
# ═══════════════════════════════════════════════════════════════
# 📊 MULTI-FEATURE REGRESSION
# ═══════════════════════════════════════════════════════════════
np.random.seed(42)
m_multi = 100

# 3 features: [Luas, Kamar, Usia]
X_multi = np.column_stack([
    np.random.randint(50, 200, m_multi),    # Luas 50-200 m²
    np.random.randint(1, 6, m_multi),        # Kamar 1-5
    np.random.randint(1, 30, m_multi)        # Usia 1-30 tahun
]).astype(float)

# True relationship: Harga = 100 + 5*Luas + 50*Kamar - 2*Usia + noise
y_multi = 100 + 5*X_multi[:,0] + 50*X_multi[:,1] - 2*X_multi[:,2] + np.random.randn(m_multi)*20
y_multi = y_multi.reshape(-1, 1)

# Scale features
scaler = StandardScaler()
X_multi_scaled = scaler.fit_transform(X_multi)
X_multi_b = np.c_[np.ones((m_multi, 1)), X_multi_scaled]

# Train
model_multi = AdvancedSGD(learning_rate=0.1, n_epochs=200, batch_size=16, momentum=0.9)
model_multi.fit(X_multi_b, y_multi)

print('📊 MULTI-FEATURE REGRESSION')
print('='*50)
print(f'Features: [Luas, Kamar, Usia]')
print(f'True coefficients: [5, 50, -2]')
print(f'Learned theta (scaled): {model_multi.theta.flatten()[1:]}')
print(f'Final cost: {model_multi.history[-1]:.4f}')
print()
print('💡 Note: Coefficients berbeda karena fitur sudah di-scale.')
print('   Untuk interpretasi, perlu inverse_transform.')


## ⚠️ BONUS 6: Common Pitfalls & Debugging

### 🚨 Masalah Umum:

| Problem | Gejala | Solusi |
|---------|--------|--------|
| **Learning Rate terlalu besar** | Loss naik / NaN / Inf | Kurangi LR (coba /10) |
| **Learning Rate terlalu kecil** | Loss turun sangat lambat | Naikkan LR (coba *10) |
| **Fitur tidak di-scale** | Konvergensi lambat, zig-zag | Pakai StandardScaler |
| **Exploding Gradients** | Loss = NaN | Gradient clipping / kurangi LR |
| **Stuck di local minimum** | Loss tidak turun lagi | Tambah momentum / restart |
| **Overfitting** | Training loss bagus, test buruk | Regularization / Early stopping |

### 🛠️ Debugging Checklist:

```python
# 1. Cek data shape
print(f'X shape: {X.shape}, y shape: {y.shape}')

# 2. Cek ada NaN/Inf
print(f'NaN in X: {np.isnan(X).any()}')
print(f'Inf in X: {np.isinf(X).any()}')

# 3. Cek gradient magnitude
print(f'Gradient norm: {np.linalg.norm(gradient)}')

# 4. Plot loss curve
plt.plot(model.history)
plt.title('Loss Curve')
plt.show()
```


## 🔗 BONUS 7: Connection to Deep Learning

**Semua yang kamu pelajari di sini adalah FONDASI untuk Neural Networks!**

| Konsep di Module Ini | Di Deep Learning |
|---------------------|------------------|
| Gradient Descent | **Sama persis** - SGD, Adam, RMSprop |
| Learning Rate | **Sama** - LR Scheduling lebih kompleks |
| Batch Size | **Sama** - Mini-batch standar industri |
| Momentum | **Sama** - Dipakai di Adam, RMSprop |
| Feature Scaling | **BatchNorm, LayerNorm** |
| Gradient Check | **Autograd** (PyTorch/TensorFlow) |
| Early Stopping | **Callbacks** di Keras/PyTorch |

**💡 Takeaway:** Kuasai SGD di Linear Regression = 50% bekal untuk Deep Learning!


## 📝 SUMMARY (JIKA-MAKA)

| Metode | Sifat | Kapan Pakai? (JIKA...) |
|--------|-------|--------------------------|
| **Batch GD** | Stabil, Lambat | Data muat di RAM, butuh presisi tinggi |
| **SGD** | Cepat, Berisik | Data SANGAT besar (streaming), butuh less memory |
| **Mini-Batch** | Balance | **Standard Industry Practice** (Default di Deep Learning) |
| **Momentum** | Cepat Konvergen | Cost function berlembah/berbelok tajam |
| **Normal Eq (inv)** | Analitik | Dataset kecil (<10k), Matrix full rank |
| **Normal Eq (pinv)** | Robust | Sda, tapi ada fitur redundan/singular |

